In [109]:
import os
import json
import pandas as pd
import traceback

In [110]:
from langchain.chat_models import ChatOpenAI

In [111]:
from dotenv import load_dotenv
load_dotenv()

True

In [112]:
KEY=os.getenv("OPENAI_API_KEY")

In [114]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [116]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [117]:
RESPONSE_JSON = {
  "1": {
    "mcq":"multiple choice question",
    "options":{
      "a":"choice here",
      "b":"choice here",
      "c":"choice here",
      "d":"choice here",
    },
    "correct":"correct answer",
  },
    "2": {
    "mcq":"multiple choice question",
    "options":{
      "a":"choice here",
      "b":"choice here",
      "c":"choice here",
      "d":"choice here",
    },
    "correct":"correct answer",
  },
    "3": {
    "mcq":"multiple choice question",
    "options":{
      "a":"choice here",
      "b":"choice here",
      "c":"choice here",
      "d":"choice here",
    },
    "correct":"correct answer",
  },
}

In [118]:
TEMPLATE="""
Text={text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz od {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be comforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [119]:
quiz_generation_prompt = PromptTemplate(
  input_variables=["text","number","subject","tone","response_json"],
  template=TEMPLATE
)

In [120]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

In [121]:
Template2="""
You are an expert english grammarian and writer.Given a Multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at per with the cognitive and analytical abilities od the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [122]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE)

In [123]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [124]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain],
                                        input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"],
                                        verbose=True,)

In [125]:
file_path =r"/Users/yingyinghao/mcqgen/data.txt"

In [126]:
with open(file_path, 'r') as file:
  TEXT =file.read

In [127]:
print(TEXT)

<built-in method read of _io.TextIOWrapper object at 0x172dc0520>


In [129]:
NUMBER = 5
SUBJECT ="machine learning"
TONE='simple'

In [130]:
#how to setup Token Usage Tracking in Langchain
with get_openai_callback() as cb:
  response=generate_evaluate_chain(
    {
      "text": TEXT,
      "number": NUMBER,
      "subject":SUBJECT,
      "tone":TONE,
      "response_json":json.dumps(RESPONSE_JSON)
    }
  )

/Users/yingyinghao/mcqgen/env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text=<built-in method read of _io.TextIOWrapper object at 0x172dc0520>
You are an expert MCQ maker. Given the above text, it is your job to create a quiz od 5 multiple choice questions for machine learning students in simple tone.
Make sure the questions are not repeated and check all the questions to be comforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.Ensure to make 5 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here",

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
quiz=response.get("quiz")

NameError: name 'response' is not defined

In [131]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total cost: {cb.total_cost}")


Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total cost: 0.0


In [132]:
quiz=response.get("quiz")

NameError: name 'response' is not defined

In [133]:
json.loads(quiz)

NameError: name 'quiz' is not defined

In [134]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
      [
        f"{option}:{option_value}"
        for option, option_value in value["options"].items()
        ]
      )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

NameError: name 'quiz' is not defined

In [135]:
pd.DataFrame(quiz_table_data)

""
